In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk

make sure to upload valueset and fhir bundle everytime your runtime shuts down

In [ ]:
df = pd.read_excel('/content/ValueSets (1).xlsx', sheet_name=1)

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

Upload and show RXnorm medication codes for OUR reference

In [ ]:
df_Rx = df[df['Code System'].str.contains('RXNORM') == True]
#df_L[df_L['Code System'] == 'LOINC']
df_Rx

,Name,Code System,Type,Steward,OID,Expansion Status,Expansion Date,Code Count
59,Abemaciclib,RXNORM,Extensional,CancerLinQ,2.16.840.1.113762.1.4.1260.476,Active,2025-09-16,24
87,Abortion Medications,RXNORM,Grouping,NACHC,2.16.840.1.113762.1.4.1221.179,Active,2025-09-16,123
141,Acamprosate,RXNORM,Extensional,NCQA,2.16.840.1.113883.3.464.1003.1154,Not Maintained,2025-09-16,1
153,ACE ARB Prescription,RXNORM,Extensional,Cliniwiz,2.16.840.1.113762.1.4.1200.245,Not Maintained,2025-09-16,284
155,ACE inhibitor allergy RXNORM,RXNORM,Extensional,Optum,2.16.840.1.113762.1.4.1078.45,Active,2025-09-16,279
...,...,...,...,...,...,...,...,...
17040,Zileuton Oral Preparations,RXNORM,Extensional,NHLBI Implementation Science and Health Care I...,2.16.840.1.113762.1.4.1196.468,Not Maintained,2025-09-16,3
17041,Zinc,RXNORM,Extensional,Clinical Architecture,2.16.840.1.113883.3.3616.200.110.102.2091,Deprecated,2023-11-20,65
17042,Ziprasidone,RXNORM,Extensional,NCQA,2.16.840.1.113883.3.464.1003.1141,Not Maintained,2025-09-16,5
17043,Zolmitriptan,RXNORM,Extensional,AAN,2.16.840.1.113762.1.4.1034.656,Active,2025-09-16,4


Upload BioBert for semantic vectors

In [ ]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, models

# 2) Build a Sentence-Transformers pipeline around Bio_ClinicalBERT
#    - 'Transformer' loads the HF model
#    - 'Pooling' does mean-pooling over token embeddings
biobert_name = "emilyalsentzer/Bio_ClinicalBERT"
word_model = models.Transformer(biobert_name, max_seq_length=256)
pooling    = models.Pooling(word_model.get_word_embedding_dimension(),
                            pooling_mode_mean_tokens=True)
model = SentenceTransformer(modules=[word_model, pooling])

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Import FHIR bundles and turn then into python dicts

In [ ]:
import glob
import json

json_dicts = []  # this will store all JSON files as Python dicts

for path in glob.glob("/content/synthetic data /*.json"):
    with open(path, "r") as f:
        data = json.load(f)   # ← this becomes a dict
        json_dicts.append(data)

# Check results
print(f"Loaded {len(json_dicts)} JSON files.")
print(type(json_dicts[0]))    # should be <class 'dict'>


Loaded 99 JSON files.
<class 'dict'>


function to turn dates into age

In [ ]:
from datetime import datetime, date

def birthday_to_age(birthdate_str):
    # Expecting format "YYYY-MM-DD"
    birthdate = datetime.strptime(birthdate_str, "%Y-%m-%d").date()
    today = date.today()

    age = today.year - birthdate.year - (
        (today.month, today.day) < (birthdate.month, birthdate.day)
    )
    return age

print(birthday_to_age("1990-04-23"))


35


Reference for extracting patient info

In [ ]:
print(json_dicts[0]['entry'][0]['resource']['birthDate'])

2004-08-05


Creating dataset, keep it anonymous to "remain compliant"

In [ ]:
data = pd.DataFrame(columns=['sex', "age", 'weight'])

Weight lives in the observation resource, medical data is really messy so it could be inside any observation, there are likely tons of them. There might be none, we will pick the latest one.

In [ ]:
from datetime import datetime

# weight is in KGs

def get_latest_weight(bundle):
    """
    Given a FHIR Bundle (as a Python dict), return the most recent
    body weight value (int/float) or None if no weight is found.
    """
    weights = []

    # Collect all weight observations with their dates
    for entry in bundle.get("entry", []):
        resource = entry.get("resource", {})

        if resource.get("resourceType") == "Observation":
            # Check if code matches body weight (LOINC 29463-7 or 3141-9)
            codings = resource.get("code", {}).get("coding", [])
            if not any(c.get("code") in ["29463-7", "3141-9"] for c in codings):
                continue

            # measurement value
            value = resource.get("valueQuantity", {}).get("value")

            # date of measurement
            date = (
                resource.get("effectiveDateTime")
                or resource.get("effectivePeriod", {}).get("start")
            )

            if value is not None and date is not None:
                weights.append((value, date))

    # No weights found
    if not weights:
        return None

    # Helper to parse ISO datetimes like "2021-02-05T00:00:00Z"
    def parse_dt(d):
        return datetime.fromisoformat(d.replace("Z", "+00:00"))

    # Pick the (value, date) pair with the latest date
    latest_value, _latest_date = max(weights, key=lambda x: parse_dt(x[1]))

    # Ensure numeric if possible, but keep int/float "as it came"
    if isinstance(latest_value, (int, float)):
        return latest_value
    try:
        return float(latest_value)
    except (TypeError, ValueError):
        return latest_value  # fallback: return as-is


get_latest_weight(json_dicts[0])

59.5

In [ ]:
sex = []
age = []
weight = []
for patient in json_dicts:
  weight.append(get_latest_weight(patient))
  patient_sex = patient['entry'][0]['resource']['gender']
  if patient_sex == 'female':
    sex.append(0)
  else:
    sex.append(1)
  age.append(birthday_to_age(patient['entry'][0]['resource']['birthDate']))

data['age'] = age
data['sex'] = sex
data['weight'] = weight
data

,sex,age,weight
0,0,21,59.5
1,1,46,130.9
2,0,11,24.4
3,1,14,58.3
4,0,66,77.3
...,...,...,...
94,0,21,51.6
95,1,18,91.1
96,1,2,14.9
97,0,34,81.7


When we add new data this might change

In [ ]:
data["weight"].isna().sum()

np.int64(0)

# Classification Step

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
df = # extracted_data

X = df.drop(columns=["medication_code"])
y = df["medication_code"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_siz=0.2, random_state=42, stratify=y)


In [ ]:
xgb = XGBClassifier(
    objective='',
    eval_metric='',
    max_depth='',
    n_estimators='',
    learning_rate='',
    subsample='',
    colsample_bytree=''
)

model = Pipeline(steps=[('preprocess', preprocess), ('xgb', xgb)])

In [ ]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))